In [1]:
import pandas as pd


import numpy as np
import pandas as pd

import seaborn as sns
from importlib import reload
import matplotlib.pyplot as plt
import preprocess_data 

reload(preprocess_data)

%matplotlib inline

In [2]:
measurements_blood_pressure = pd.read_csv('/data/public/MLA/VCHAMPS-Quality/measurements_blood_pressure_qual.csv').iloc[:,1:]

In [3]:
path = '/home/hassan/lily/MLA/FDA/inpatient_simple_quality.csv'
inpatient = pd.read_csv(path).iloc[:,1:]
inpatient_date = inpatient[['Internalpatientid','Discharge date']]
inpatient_date

,Internalpatientid,Discharge date
0,67,2019-04-29 22:39:20.0
1,200,2019-01-31 22:59:02.0
2,291,2006-07-08 13:56:24.0
3,330,2019-06-28 14:50:58.0
4,351,2023-07-10 21:33:42.0
...,...,...
627,166881,2001-01-09 21:07:54.0
628,167102,2011-11-27 12:57:02.0
629,167404,2004-01-30 11:12:01.0
630,167917,2016-08-07 11:25:36.0


In [4]:
# Filter procedure date that's before discharge date
measurements_blood_pressure = measurements_blood_pressure.merge(inpatient_date, how = 'left', on = 'Internalpatientid')
measurements_blood_pressure = measurements_blood_pressure.loc[measurements_blood_pressure['Measurement date'] <= measurements_blood_pressure['Discharge date']]
measurements_blood_pressure 

,Internalpatientid,Age at measurement bp,Measurement date,Diastolic bp,Systolic bp,State,Discharge date
0,100012,53.086669,2002-08-02 04:15:26.0,75.0,137.0,New Mexico,2007-08-27 10:48:05.0
1,100012,53.583655,2003-01-30 19:49:29.0,87.0,161.0,New Mexico,2007-08-27 10:48:05.0
2,100012,53.837326,2003-05-03 13:01:16.0,77.0,144.0,New Mexico,2007-08-27 10:48:05.0
3,100012,53.898581,2003-05-25 22:21:03.0,73.0,136.0,New Mexico,2007-08-27 10:48:05.0
4,100012,54.044102,2003-07-18 02:51:48.0,68.0,143.0,New Mexico,2007-08-27 10:48:05.0
...,...,...,...,...,...,...,...
192313,99870,87.526241,2008-12-23 20:11:01.0,84.0,140.0,South Dakota,2008-12-27 10:57:14.0
192314,99870,87.532475,2008-12-26 02:52:19.0,92.0,159.0,South Dakota,2008-12-27 10:57:14.0
192315,99870,87.534656,2008-12-26 22:00:11.0,80.0,153.0,South Dakota,2008-12-27 10:57:14.0
192317,99941,67.253361,2009-10-24 02:28:58.0,62.0,151.0,Maine,2010-11-28 13:49:20.0


In [5]:
previous_df = inpatient_date
bins = [0, 100, 130, 200]

for i in range(1,2):
    data = measurements_blood_pressure['Internalpatientid'].loc[(measurements_blood_pressure['Systolic bp'] > i) & (measurements_blood_pressure['Age at measurement bp'] <= (i+1)*20)]
    new_df = pd.crosstab(data, pd.cut(measurements_blood_pressure['Systolic bp'], bins)).reset_index().fillna(0)
    new_df = new_df.rename(columns={pd.Interval(0, 60, closed='right'): 'Age ' + str(i*20) + '-' + str((i+1)*20)+ ' hypotension',
                                    pd.Interval(60, 80, closed='right'): 'Age ' + str(i*20) + '-' + str((i+1)*20) + ' healthy',
                                    pd.Interval(80, 200, closed='right'): 'Age ' + str(i*20) + '-' + str((i+1)*20) + ' hypetension'})
    previous_df = previous_df.merge(new_df, how = 'left', on = 'Internalpatientid')


In [6]:
# hypotension
bins = [20,40,60,80,100, 120]
previous_df = inpatient_date
data = measurements_blood_pressure['Internalpatientid'].loc[(measurements_blood_pressure['Systolic bp'] < 100) | (measurements_blood_pressure['Diastolic bp'] < 60)]
new_df = pd.crosstab(data, pd.cut(measurements_blood_pressure['Age at measurement bp'], bins)).reset_index().fillna(0)
new_df = new_df.rename(columns={pd.Interval(20, 40, closed='right'): 'Age 20-40 hypotension',
                                pd.Interval(40, 60, closed='right'): 'Age 40-60 hypotension',
                                pd.Interval(60, 80, closed='right'): 'Age 60-80 hypotension',
                                pd.Interval(80, 100, closed='right'): 'Age 80-100 hypotension',
                                pd.Interval(100, 120, closed='right'): 'Age 100-120 hypotension'})
previous_df = previous_df.merge(new_df, how = 'left', on = 'Internalpatientid')

In [7]:
# healthy
data = measurements_blood_pressure['Internalpatientid'].loc[(measurements_blood_pressure['Systolic bp'] >= 100) & (measurements_blood_pressure['Systolic bp'] < 130) 
                                                            & (measurements_blood_pressure['Diastolic bp'] >= 60) & (measurements_blood_pressure['Diastolic bp'] < 80)]
new_df = pd.crosstab(data, pd.cut(measurements_blood_pressure['Age at measurement bp'], bins)).reset_index().fillna(0)
new_df = new_df.rename(columns={pd.Interval(20, 40, closed='right'): 'Age 20-40 healthy',
                                pd.Interval(40, 60, closed='right'): 'Age 40-60 healthy',
                                pd.Interval(60, 80, closed='right'): 'Age 60-80 healthy',
                                pd.Interval(80, 100, closed='right'): 'Age 80-100 healthy',
                                pd.Interval(100, 120, closed='right'): 'Age 100-120 healthy'})
previous_df = previous_df.merge(new_df, how = 'left', on = 'Internalpatientid')

In [8]:
# hypertension
data = measurements_blood_pressure['Internalpatientid'].loc[(measurements_blood_pressure['Systolic bp'] > 130) | (measurements_blood_pressure['Diastolic bp'] > 80) ]
new_df = pd.crosstab(data, pd.cut(measurements_blood_pressure['Age at measurement bp'], bins)).reset_index().fillna(0)
new_df = new_df.rename(columns={pd.Interval(20, 40, closed='right'): 'Age 20-40 hypertension',
                                pd.Interval(40, 60, closed='right'): 'Age 40-60 hypertension',
                                pd.Interval(60, 80, closed='right'): 'Age 60-80 hypertension',
                                pd.Interval(80, 100, closed='right'): 'Age 80-100 hypertension',
                                pd.Interval(100, 120, closed='right'): 'Age 100-120 hypertension'})
previous_df = previous_df.merge(new_df, how = 'left', on = 'Internalpatientid')

In [9]:
previous_df = previous_df.drop(columns = ['Discharge date'])
previous_df.head()

,Internalpatientid,Age 20-40 hypotension,Age 40-60 hypotension,Age 60-80 hypotension,Age 80-100 hypotension,Age 100-120 hypotension,Age 20-40 healthy,Age 40-60 healthy,Age 60-80 healthy,Age 80-100 healthy,Age 100-120 healthy,Age 20-40 hypertension,Age 40-60 hypertension,Age 60-80 hypertension,Age 80-100 hypertension,Age 100-120 hypertension
0,67,NaN,NaN,NaN,NaN,NaN,0.0,3.0,0.0,0.0,0.0,0.0,69.0,0.0,0.0,0.0
1,200,0.0,0.0,10.0,5.0,0.0,0.0,0.0,41.0,10.0,0.0,0.0,0.0,68.0,21.0,0.0
2,291,0.0,0.0,0.0,53.0,0.0,0.0,0.0,0.0,13.0,0.0,0.0,0.0,0.0,57.0,0.0
3,330,0.0,0.0,29.0,0.0,0.0,0.0,5.0,72.0,0.0,0.0,0.0,4.0,16.0,0.0,0.0
4,351,0.0,0.0,0.0,35.0,0.0,0.0,0.0,0.0,15.0,0.0,0.0,0.0,0.0,81.0,0.0


In [10]:
previous_df.to_csv('/home/vivi/FDA_datasets/inpatient_measurements_bp_quality_admission.csv')